In [4]:
import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    Create a dataframe to be used as the base of a power prediction model
'''

datadir = "/datasets/eurora_data/db1/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-11-01')
train_start_time = pd.to_datetime('2014-03-31')
train_end_time = pd.to_datetime('2014-10-01')
test_end_time = pd.to_datetime('2014-11-01')


infile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [7]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs = pd.read_csv(infile_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(jobs.shape[0], interval_comment))

# clean user names
jobs['user'] = jobs.apply(lambda row: row['user'].split('@')[0], axis=1)

jobs_to_nodes_whole_data contains 469095 records


In [3]:
long_jobs_q1 = jobs[jobs['real_pow_quality'] > 0]
long_jobs_q1 = long_jobs_q1[long_jobs_q1['real_pow'] > 0]

#jobs_q1 = jobs[jobs['real_pow_quality'] > 0]
#jobs_q1 = jobs_q1[jobs_q1['real_pow'] > 0]

jobs_train = long_jobs_q1[pd.to_datetime(long_jobs_q1['end_time']) <= train_end_time]
jobs_train = jobs_train[pd.to_datetime(jobs_train['run_start_time']) > train_start_time]


print("train set contains {} records".format(jobs_train.shape[0]))

jobs_test = jobs[pd.to_datetime(jobs['end_time']) <= test_end_time]
jobs_test = jobs_test[pd.to_datetime(jobs_test['run_start_time']) > train_end_time]
print("test set contains {} records".format(jobs_test.shape[0]))

train set contains 146901 records
test set contains 29935 records


In [ ]:
###### NB eseguire solo se non si vogliono job sotto 5 sec

jobs_train = jobs_train[pd.to_datetime(jobs_train['end_time']) - pd.to_datetime(jobs_train['run_start_time']) > np.timedelta64(5, 's')]
print(jobs_train.shape[0])
jobs_test = jobs_test[pd.to_datetime(jobs_test['end_time']) - pd.to_datetime(jobs_test['run_start_time']) > np.timedelta64(5, 's')]
print(jobs_test.shape[0])

In [6]:
# train_whole: 167324 ; test_whole:29935 
# train_long: 39363 ; test_long: 11622

In [10]:
### clean the data
# remove jobs runned on the inexistent node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 
# remove jobs with the same id that runned on the same node
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

### merge the info from the two tables
merged_jobs_to_nodes = pd.merge(jobs_train, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("merged_jobs_to_nodes contains {} records".format(merged_jobs_to_nodes.shape[0]))

after the clean, jobs_to_nodes_whole_data contains 444610 records
merged_jobs_to_nodes contains 52396 records


In [11]:
jobs_to_nodes_whole_data.head(1)

,node_id,job_id_string,ncpus,ngpus,nmics,mem_requested
job2node_id,,,,,,
1,17,498458.node129,16,0,0,14680064


In [12]:
jobs_train.head(1)

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,mic_req,real_pow,ran_alone,real_pow_quality,n_2_1,n_2_2,n_3_1,job_tot_timepoints,job_timepoints,good_nodes
job_id,,,,,,,,,,,,,,,,,,,,,
1,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000,1,16,14,...,NaN,0.0,True,1.0,0,1,0,631,631,0


In [13]:
#queues = long_jobs['queue'].unique()
#nodes_labels = ['node_%s'%(x) for x in range(1, 64)]
#nodes_counters = ['node_count_%s'%(x) for x in range(1, 64)]

#nodes_labels + nodes_counters

In [14]:
## node dataframe
#index_node = jobs_to_nodes_whole_data.groupby(['node_id']).size().reset_index(name='counts')
#columns_node = ['node_id', 'counts', 'avg']
#df_node = pd.DataFrame(index_node, index=range(0, index_node.shape[0]), columns=columns_node)
#df_node.head(3)

In [15]:
## user dataframe
#index_user = long_jobs.groupby(['user']).size().reset_index(name='counts')
#columns_user = ['user', 'counts', 'avg']
#df_user = pd.DataFrame(index_user, index=range(0, index_user.shape[0]), columns=columns_user)
#df_user.head(3)

In [16]:
## user job dataframe
#index_user_node = merged_jobs_to_nodes.groupby(['user', 'node_id']).size().reset_index(name='counts')
#columns_user_job = ['user', 'node_id', 'counts', 'avg']
#df_user_node = pd.DataFrame(index_user_node, index=range(0, index_user_node.shape[0]), columns=columns_user_job)
#df_user_node.head(3)

In [17]:
## user dataframe
#index_user_job = merged_jobs_to_nodes.groupby(['user', 'job_name']).size().reset_index(name='counts')
#columns_user_job = ['user', 'job_name', 'counts', 'avg']
#df_user_job = pd.DataFrame(index_user_job, index=range(0, index_user_job.shape[0]), columns=columns_user_job)
#df_user_job.head(3)

In [18]:
#index_user_job_node = merged_jobs_to_nodes.groupby(['user', 'job_name', 'node_id']).size().reset_index(name='counts')
#columns_user_job_node = ['user', 'job_name', 'node_id', 'counts', 'avg']
#df_user_job_node = pd.DataFrame(index_user_job_node, index=range(0, index_user_job_node.shape[0]), columns=columns_user_job_node)
#df_user_job_node.head(3)

In [19]:
#print(df_node.shape[0])
#print(df_user.shape[0])
#print(df_user_node.shape[0])
#print(df_user_job.shape[0])
#print(df_user_job_node.shape[0])

### good code

We want to obtain 5 dataframes:
- node
- user
- user_node
- user_job
- user_job_node

In [20]:
merged_jobs_to_nodes['core_avg'] = merged_jobs_to_nodes['real_pow'] / merged_jobs_to_nodes['cpu_req']

In [21]:
# user_jobName_node
df_user_job_node = merged_jobs_to_nodes.groupby(['user', 'job_name', 'node_id']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_user_job_node.shape[0])

# Drop a row by condition
df_user_job_node = df_user_job_node[df_user_job_node['mean_core'] != 0.0]
df_user_job_node.head(100)

21627


,user,job_name,node_id,count,mean_core
0,a07smr01,A2REPO,7,1,12.412156
1,a07smr01,A2REPO,33,1,3.937672
2,a07smr01,A2REPO,35,59,4.375295
3,a07smr01,A2REPO,46,24,2.406068
4,a07smr01,ARCHIVE,7,1,11.895912
5,a07smr01,ARKI,33,7,5.766090
6,a07smr01,ARKI,35,81,4.299856
7,a07smr01,ARKI,46,27,3.992453
8,a07smr01,INIT,33,4,4.631090
9,a07smr01,INIT,35,78,3.525155


In [22]:
#outfile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole"
outfile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole_long"
df_user_job_node.to_csv(outfile_user_job_node + ".csv")

In [23]:
# user_jobName
df_user_job = merged_jobs_to_nodes.groupby(['user', 'job_name']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_user_job.shape[0])
# Drop a row by condition
df_user_job = df_user_job[df_user_job['mean_core'] != 0.0]
df_user_job.head(5)

6351


,user,job_name,count,mean_core
0,a07smr01,A2REPO,85,3.908680
1,a07smr01,ARCHIVE,1,11.895912
2,a07smr01,ARKI,115,4.316932
3,a07smr01,INIT,109,4.240230
4,a07smr01,LM28_OPE,1218,21.279671


In [24]:
# outfile_user_job = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_whole"
outfile_user_job = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_whole_long"
df_user_job.to_csv(outfile_user_job + ".csv")

In [25]:
# user_node
df_user_node = merged_jobs_to_nodes.groupby(['user', 'node_id']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_user_node.shape[0])
# Drop a row by condition
df_user_node = df_user_node[df_user_node['mean_core'] != 0.0]
df_user_node.head(5)

4684


,user,node_id,count,mean_core
0,a07smr01,1,5,15.386652
1,a07smr01,2,4,18.405537
2,a07smr01,6,4,18.405537
3,a07smr01,7,3,28.234849
4,a07smr01,33,40,7.076942


In [26]:
#outfile_user_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_node_whole"
outfile_user_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_node_whole_long"
df_user_node.to_csv(outfile_user_node + ".csv")

In [27]:
# user
df_user = merged_jobs_to_nodes.groupby(['user']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_user.shape[0])
# Drop a row by condition
df_user = df_user[df_user['mean_core'] != 0.0]
df_user.head(5)

248


,user,count,mean_core
0,a07smr01,6108,15.214226
1,a08tra01,2,30.078129
2,a08tra39,1,88.137393
3,a08tra41,2,44.349381
4,a08tra59,5,30.123967


In [28]:
#outfile_user = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_whole"
outfile_user = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_whole_long"
df_user.to_csv(outfile_user + ".csv")

In [29]:
# node
df_node = merged_jobs_to_nodes.groupby(['node_id']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_node.shape[0])
# Drop a row by condition
df_node = df_node[df_node['mean_core'] != 0.0]
df_node.head(5)

63


,node_id,count,mean_core
0,1,576,10.522231
1,2,98,9.240703
2,3,718,17.432847
3,4,847,17.167538
4,5,346,19.628440


In [30]:
#outfile_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_node_whole"
outfile_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_node_whole_long"
df_node.to_csv(outfile_node + ".csv")